In [1]:
import getpass
import ipywidgets as ipw
import os
import json
import shlex
import io
import re
import tempfile
import subprocess
import numpy as np
import csv
import warnings
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from IPython.display import FileLink

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
tf.logging.set_verbosity(tf.logging.ERROR)


/home/shared/NRG/mmilchenko/lib/python3-tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shared/NRG/mmilchenko/lib/python3-tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shared/NRG/mmilchenko/lib/python3-tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

In [2]:
class ServerParams:
    '''
    Container parameters received from XNAT
    '''
    def __init__(self,server=None, user=None, password=None, project=None,subject=None,experiment=None):
        self.server,self.user,self.password,self.project,self.subject,self.experiment= \
            server,user,password,project,subject,experiment
        self.jsession=''
        self.connected=False

    def __str__(self):
        return "server:{}, user: {}, project: {}, subject: {}, experiment: {}, connected: {}".\
            format(self.server,self.user,self.project,self.subject,self.experiment,self.connected)
        
    def connect(self):
        cmd="curl -k -u "+ self.user+":"+self.password+ \
            " "+self.server+"/data/JSESSION"        
        self.jsession=os.popen(cmd).read()
        self.connected=(len(self.jsession)==32)
        return self.connected
    
class XnatIterator:
    def __init__(self,sp):
        self.sp=sp
        self._subjects=[]
        self._experiments=[]
        self._scans=[]
            
    def _curl_cmd_prefix(self):
        return "curl  -k --cookie JSESSIONID=" + self.sp.jsession
    
    def _curl_cmd_path(self,path):
        return shlex.quote(self.sp.server+"/data/archive/projects/"+self.sp.project+path)
    
    def _curl_cmd(self,path):        
        cmd=self._curl_cmd_prefix()+' '+self._curl_cmd_path(path)
        out=os.popen(cmd).read()
        return(out)
        
    def curl_download_single_file(self,path,dest):
        cmd=self._curl_cmd_prefix()+' -o '+dest+' '+ self.sp.server + path
        return os.popen(cmd).read()
        
    def set_project(self,pr):
        self.sp.project=pr
    
    def list_subjects(self):
        tq=self._curl_cmd('/subjects?format=json')
        try: 
            df=json.loads(tq)
        except:
            return []

        subjs=sorted(df['ResultSet']['Result'], key=lambda k:k['label'])        
        self._subjects=[f['label'] for f in subjs]        
        return self._subjects
    
    def scan_file_loader(self,scans,tdir,lock):
        for s in scans:
            #print(s)
            files=self.list_scan_files(s['subject'],s['experiment'],s['ID'])
            if len(files)>0:
                t=tdir+'/'+s['subject']+'_'+s['experiment']+'_'+s['ID']
                self.curl_download_single_file(files[0],t+'.dcm')
                os.system("dcmj2pnm +G +Wn +on "+t+".dcm "+ t + ".png")
                os.system( "rm -f " + t + ".dcm" )
                lock.acquire()
                s['png'] = t+".png"
                lock.release()
            else:
                s['png']='N/A'
    
    def list_experiments(self,subject):
        tq=self._curl_cmd('/subjects/'+subject+"/experiments?xsiType=xnat:imageSessionData&format=json") 
        try:
            df=json.loads(tq)
        except:
            print ('error listing experiments!')
            return []
        exps=sorted(df['ResultSet']['Result'], key=lambda k:k['date'])
        self._experiments=[f['label'] for f in exps]
        return self._experiments
    
    def list_scans(self,subject,experiment, listDcmFiles=False):
        sf=self._curl_cmd('/subjects/'+ subject +'/experiments/' \
            +experiment + "/scans?columns=ID,frames,type,series_description")
        try: 
            df=json.loads(sf)
        except:
            return []
        self._scans=sorted(df['ResultSet']['Result'], key=lambda k:k['xnat_imagescandata_id'])
        for s in self._scans:            
            s['subject']=subject
            s['experiment']=experiment
        
        if listDcmFiles:
            for s in self._scans:
                files=self.list_scan_files(subject,experiment,s['ID'])
                s['files']=files
        return self._scans
    
    def get_dcm_files_for_scans(self,subject,experiment,scans):
        for s in scans:
            files=self.list_scan_files(subject,experiment,s['ID'])
            s['files']=files
        
    def list_scan_files(self,subject,experiment,scan):
        sf=self._curl_cmd('/subjects/'+ subject +'/experiments/' \
            +experiment + '/scans/'+scan+'/resources/DICOM/files')
        try: df=json.loads(sf)
        except:
            return []
        lst=sorted(df['ResultSet']['Result'], key=lambda k:k['Name'])
        return [ f['URI'] for f in lst ]
    """
    list all scans in project, filtered by subject prefix. 
    Display progres in output textarea.
    Save output in speficified json file.
    """
    def list_scans_all(self,subjects,subject_prefix,output):
        scans=[]
        ns=0
        for su in subjects:
            if not su.lower().startswith(subject_prefix.lower()): continue
            experiments=self.list_experiments(su)
            for e in experiments:
                if output: output.value='running, found {} scans'.format(ns)
                sscans=self.list_scans(su,e)
                for s in sscans:
                    scans.append(s)
                    ns+=1
        return scans
    
class HOF_Classifier:
    def __init__(self):
        self.classifier=[]
        self.vectorizer=[]
        self._class_vectorizer=None
        #important: must be in aphpabetical order for vectorizer to work correctly.
        self._classes=['CBF','CBV','DSC','DWI','FA','MD','MPRAGE','MTT','OT','PBP','SWI','T1hi','T1lo','T2FLAIR','T2hi','T2lo','TRACEW','TTP']
        #self._scan_list=[]
    def load_json(self, json_file):
        with open(json_file, 'r') as fp:
            out_dict=json.loads(fp.read())
        return out_dict    
    def save_json(self, var, file):
        with open(file,'w') as fp:
            json.dump(var, fp) 
    '''
    Assign HOF ID's to scans using associative table look-up.
    '''
    def assign_hofids_slist(self,scans):
        for s in scans:
            descr=re.sub(' ','',s['series_description'])
            cmd="slist qd "+"\"" + descr + "\""
            try:
                hof_id=os.popen(cmd).read().split()[1]
            except:
                hof_id=""
            #print(hof_id)
            s['hof_id']=hof_id
            #out.value="{}/{}".format(s['series_description'],hof_id)
        return scans
    
    def write_scans_csv(self, scans, file):
        with open(file, 'w') as output_file:
            dict_writer = csv.DictWriter(output_file, scans[0].keys())
            dict_writer.writeheader()
            dict_writer.writerows(scans)
            
    def read_scans_csv(self, file):
        with open(file,'r') as inf:
            reader = csv.DictReader(inf)
            scans=[{k: str(v) for k,v in row.items()} 
                      for row in csv.DictReader(inf,skipinitialspace=True)]
        return scans    
       
    '''
    Create vocabulary from the bag of words. These will act as features.
    '''
    def gen_vocabulary(self,scans):
        descs=self.prepare_descs(scans)
        vectorizer=CountVectorizer(min_df=0)
        vectorizer.fit(descs)
        self.vectorizer=vectorizer
        print('the length of vocabulary is ',len(vectorizer.vocabulary_))
     
    #for logreg/svm output, categorical labels are stored as strings
    def prepare_training_vectors(self,scans):
        #labels vector.
        vectorized_descs=self.gen_bow_vectors(scans)
        y=[ s['hof_id'] for s in scans ]
        return vectorized_descs,y
    
    #for a NN output, categorical labels are stored as BOW over vocabulary of class labels.
    def prepare_training_vectors_nn(self,scans,gen_hofids=True):
        if self._class_vectorizer is None:
            vectorizer=CountVectorizer(min_df=0)
            vectorizer.fit(self._classes)
            self._class_vectorizer=vectorizer
        vectorizer=self._class_vectorizer
        vectorized_descs=self.gen_bow_vectors(scans)
        hofids=[ s['hof_id'] for s in scans ] if gen_hofids else []        
        return vectorized_descs,vectorizer.transform(hofids).toarray()
    
    def prepare_descs(self,scans):
        #descs are 'sentences' that contain series description and log-compressed number of frames.
        descs=[]
        for s in scans:
            desc=(re.sub('[^0-9a-zA-Z ]+',' ',s['series_description'])).split()
            #compressed representation of the number of frames.
            try:
                frames='frames{}'.format(str(int(np.around(np.log(1.0+float(s['frames']))*3.0))))
            except:
                frames='frames0'
            desc.append(frames)
            descs.append(' '.join([s for s in desc if ((not s.isdigit()) and (len(s)>1)) ]))
        return descs
        
    def gen_bow_vectors(self,scans):
        if not self.vectorizer: return []
        descs=self.prepare_descs(scans)
        return self.vectorizer.transform(descs).toarray()    
    
    def train_nn(self,X,y,test_split,epochs=10,batch_size=10):
        X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=test_split,random_state=1000)
        input_dim=X_train.shape[1]
        print('input_dim:',input_dim)
        model = Sequential()
        model.add(layers.Dense(36,input_dim=input_dim,activation='relu'))
        #model.add(layers.Dense(18,activation='relu'))
        model.add(layers.Dense(len(self._classes),activation='sigmoid'))
        print('output_dim:',len(self._classes))
        model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy','categorical_accuracy'])
        model.summary()
        self.classifier=model
        #self.classifier.fit(X_train,y_train,epochs=10,verbose=True,validation_data=(X_test,y_test),batch_size=10)
        hist=self.classifier.fit(X_train,y_train,epochs=epochs,verbose=True,validation_data=(X_test,y_test),batch_size=batch_size)
        self.plot_nn_train_history(hist)
        
    def plot_nn_train_history(self,history):
        acc = history.history['acc']
        val_acc = history.history['val_acc']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        x = range(1, len(acc) + 1)

        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(x, acc, 'b', label='Training acc')
        plt.plot(x, val_acc, 'r', label='Validation acc')
        plt.title('Training and validation accuracy')
        plt.legend()
        plt.subplot(1, 2, 2)
        plt.plot(x, loss, 'b', label='Training loss')
        plt.plot(x, val_loss, 'r', label='Validation loss')
        plt.title('Training and validation loss')
        plt.legend()
        
    def infer_nn(self,scans):
        vecs,ids=self.prepare_training_vectors_nn(scans,False)
        y_fit=self.classifier.predict(vecs)        
        hofids=[ self._classes[np.argmax(y_fit[i])] for i in range(len(y_fit)) ]
        return hofids        
        
    def train_classifier(self,X,y,test_split):
        descs_train,descs_test,y_train,y_test=train_test_split(X,y,test_size=test_split,random_state=1000)
        #classifier=LogisticRegression()
        classifier=LinearSVC()
        #classifier=SVC()
        classifier.fit(descs_train,y_train)
        scoreTest=classifier.score(descs_test,y_test)
        scoreTrain=classifier.score(descs_train,y_train)
        print('Test accuracy:', scoreTest, " train accuracy:",scoreTrain)        
        self.classifier=classifier
        
        return classifier
    
    def _merge_hofids(self,scans,hofids):
        for s in scans:
            descr=re.sub(' ','',s['series_description'])
            cmd="slist qd "+"\"" + descr + "\""
            try:
                hof_id=os.popen(cmd).read().split()[1]
            except:
                hof_id=""
            #print(hof_id)
            s['hof_id']=hof_id
            out.value="{}/{}".format(s['series_description'],hof_id)
        
    def _predict_classifier(self,X):
        if not self.classifier: return []
        return self.classifier.predict(X)
        
    def predict_classifier(self, scans):
        vectorized_descs=self.gen_bow_vectors(scans)
        labels=self._predict_classifier(vectorized_descs)
        for i,s in enumerate(scans):
            s['hof_id']=labels[i]
        return scans
    
    def is_valid_model(self):
        return (self.vectorizer and self.classifier)    
        
    def save_model_nn(self,rt):
        pickle.dump(self.vectorizer,open(rt+'.vec','wb'))
        self.classifier.save(rt+'.hd5')
        
    def load_model_nn(self,rt):
        self.vectorizer=pickle.load(open(rt+'.vec','rb'))
        self.classifier=tf.keras.models.load_model(rt+'.hd5')
    
    def save_model(self, file):
        pickle.dump([self.vectorizer,self.classifier],open(file,'wb'))
                    
    def load_model(self, file):
        self.vectorizer,self.classifier=pickle.load(open(file,'rb'))    
    

In [3]:
class GUIPage():
    def __init__(self, parent, title, page_num, max_page, btn1_label=None, btn2_label=None, frontdesk=None,plumbing=None):
        style={'description_width':'initial'}
        self._parent_box,self.frontdesk,self.plumbing=parent,frontdesk,plumbing
        self._title, self._btn1_label, self._btn2_label, self._page_num, self._max_page=title, \
            btn1_label, btn2_label, page_num, max_page
        self._html_title=ipw.HTML(value='<h4>'+title+'</h4>')
#        (value=title,style={'description_width':'initial','font-size':'small'},layout={'width':'800px'})
        self.main_box=ipw.VBox([])
        self._lb_dis, self._rb_dis=False,False
        if page_num==0: self._lb_dis=True
        if page_num==max_page-1: self._rb_dis=True
        if btn1_label is None: btn1_label='Prev'
        if btn2_label is None: btn2_label='Next'
            
        self._prev_btn=ipw.Button(description=btn1_label,tooltip=str(page_num),disabled=self._lb_dis,layout={'width':'200px'})
        self._next_btn=ipw.Button(description=btn2_label,tooltip=str(page_num),disabled=self._rb_dis,layout={'width':'200px'})
        
        self._btm_indent_img=ipw.Image(width=1,height=50,layout={'width':'1px','height':'100px'})    
        self._nav_box=ipw.HBox([self._prev_btn,self._next_btn])
        self._btm_box=ipw.VBox([self._btm_indent_img,self._nav_box])
        
        if not frontdesk is None:
            self.main_box.children=[frontdesk.main_box]
    
    def show(self):
        self._parent_box.children=[self._html_title,self.main_box,self._btm_box]
        if not self.frontdesk is None:
            self.frontdesk.refresh()

class GUIBook():
    def __init__(self, pages):
        self._num_pages=len(pages)
        self.main_box=ipw.VBox()
        self.pages=[]
        for i in range(self._num_pages):
            p=pages[i]
            fd=p['frontdesk']
            
            pg=GUIPage(self.main_box,p['title'],i,self._num_pages,
                       btn1_label=p['prev_label'],btn2_label=p['next_label'],
                       frontdesk=fd,plumbing=p['plumbing'])
            if fd is not None: fd.set_nav_page(pg)
                
            self.pages+=[pg]
            pg._prev_btn.on_click(self._prev_click)
            pg._next_btn.on_click(self._next_click)
            
        if self._num_pages>0:            
            self._cur_page=0
            self.pages[0].show()
        display(self.main_box)
        
    def _prev_click(self,b):
        if self._cur_page==0: return
        self._cur_page-=1
        self.pages[self._cur_page].show()
    
    def _next_click(self,b):
        if self._cur_page==self._num_pages-1: return
        self._cur_page+=1
        self.pages[self._cur_page].show()          
        
class FrontDesk:
    def set_nav_page(self,pg):
        self._nav_page=pg
        
    def enable_nav_next(self,enable):
        self._nav_page._next_btn.disabled=not enable
        
    def enable_nav_prev(self,enable):
        self._nav_page._prev_btn.disabled=not enable
        
class XNATLogin(FrontDesk):
    def __init__(self):
        self._connected=False
        st={'description_width':'initial'}
        layout=ipw.Layout(margin='0 100pt 0 0')
        layout1=ipw.Layout(justify_content='center')
        
        self.text1=ipw.Text(value='https://xnat-dev-mga1.nrg.wustl.edu', description='XNAT server:', 
                            layout={'width':'200pt'}, style=st, disabled=False)
#        self.text1=ipw.Text(value='https://cnda.wustl.edu', description='XNAT server:', 
#                            layout={'width':'200pt'}, style=st, disabled=False)

        self.text2=ipw.Text(value='admin',description='user:',
                                disabled=False, style=st, layout={'width':'120pt'})
        self.text3=ipw.Password(value='admin',description='password:',
                                disabled=False, style=st, layout={'width':'120pt'})
        self.lbl1=ipw.Label('status: not connected', layout={'width':'120pt'}, style=st) #layout={'width':'240px','justify-content':'center'}
        lbl2=ipw.Label('',layout={'width':'120pt'},style=st)
        self.btn1=ipw.Button(description="connect",style={},layout={'width':'200pt'})
        self.btn1.on_click(self.on_connect)
        vb1=ipw.HBox([self.text1,self.text2,self.text3])
        vb2=ipw.HBox([self.btn1,lbl2,self.lbl1])
        self.main_box=ipw.VBox([vb1,vb2])
        self.sp=ServerParams()        
        
    def refresh(self):
        self.enable_nav_next(False)
        
    def on_connect(self,b):
        #self._show_scanview(False)
        self.lbl1.value='status: connecting...'
        self.sp.server,self.sp.user,self.sp.password=self.text1.value,self.text2.value,self.text3.value
        if self.sp.connect():                    
            self.lbl1.value='status: connected'
            self.btn1.description='Reconnect'            
            self.connected=True
            self.enable_nav_next(True)
        else:
            self.lbl1.value='status: connection failed'    
            self.enable_nav_next(False)
                

In [6]:
class ScanClassifierLauncher(FrontDesk):
    def __init__(self,sp):
        self._xi=XnatIterator(sp)        
        btn_lay={'width':'200pt'}
        self._fupl=ipw.FileUpload(accept='.csv',multiple=False)
        self._fupl.observe(self.read_uploaded_file)
        self._fupl_label=ipw.Label(value='Status: waiting to upload csv',layout=btn_lay)
        self._fupl_drop=ipw.Dropdown(options=['experiments','scans-raw','scans-classified'],
                                     value='scans-raw',description='csv contents',disabled=False, layout=btn_lay)
        self._fupl_box=ipw.HBox([self._fupl_drop, self._fupl, self._fupl_label])
                
        self._coll_btn=ipw.Button(description='Collect scans',layout=btn_lay)
        self._coll_btn.on_click(self.collect_scans)
        self._coll_status=ipw.Label(value='Status: ready to run.')
        self._coll_box=ipw.HBox([self._coll_btn,self._coll_status])
        
        self._out_lnk=ipw.Output()        
        
        self._classify_btn=ipw.Button(description='Classify scans',layout=btn_lay)
        self._classify_btn.on_click(self.run_classifier)
        self._classify_lbl=ipw.Label(value='Status: ready')
        self._classify_lnk=ipw.Output()
        self._classify_box=ipw.VBox([ipw.HBox([self._classify_btn,self._classify_lbl]),self._classify_lnk])

        self._group_btn=ipw.Button(description='Group scans by experiment',layout=btn_lay)
        self._group_btn.on_click(self.group_scans)
        self._group_lbl=ipw.Label(value='Status: ready')
        self._group_lnk=ipw.Output()
        self._group_box=ipw.VBox([ipw.HBox([self._group_btn,self._group_lbl]),self._group_lnk])

        self._detect_gad_btn=ipw.Button(description='Detect GAD in T1hi')
        self._detect_gad_btn.on_click(self.run_detect_gad)
                        
        self.main_box=ipw.VBox([self._fupl_box,self._coll_box,self._out_lnk,self._classify_box,self._group_box])
        
        hc=HOF_Classifier()
        hc.load_model_nn('./scan_classifier_nn.11.26.2019')
        self.hof_classifier=hc
        
        
    def write_scans_csv(self, scans, file):
        with open(file, 'w') as output_file:
            dict_writer = csv.DictWriter(output_file, scans[0].keys())
            dict_writer.writeheader()
            dict_writer.writerows(scans)
            
    def read_uploaded_file(self,b):
        fupl=self._fupl
        if not bool(fupl): return False
        keys=list(fupl.value)        
        try:
            csv_reader = csv.DictReader(io.TextIOWrapper(io.BytesIO(fupl.value[keys[0]]['content'])),skipinitialspace=True)
            self._rows=[{k: str(v) for k,v in row.items()} for row in csv_reader]
        except:
            self._fupl_label.value='Status: cannot parse csv'
            return False
        
        self._fupl_label.value='Status: csv loaded with {} rows'.format(len(self._rows))
        if self._fupl_drop.value=='scans-raw' or self._fupl_drop.value=='scans-classified':
            self.scans=self._rows
        #print(self._exps)
        return True
    
    def refresh(self):
        self.enable_nav_prev(False)
    
    def show_file_link(self,out,file):
        out.outputs=();  f=FileLink(file)
        with out:
            display(f)
            
    def group_scans(self,b):
        def add_val(dic,key,val):
            if key in dic: dic[key]+=[val]
            else: dic[key]=[val]
                
        def add_default_vals(dic,keys):
            for key in keys:
                if key not in dic: dic[key]=[]
                    
        scans=self.scans
        d={} #experiments
        def_keys=['T1c','T1nc','T2','T2FLAIR']
        def_keys_imp=['T1c_imp','T1nc_imp','T2_imp','T2FLAIR_imp']
        
        confs={} #runtime configurations, keyed by subject. 
        for r in self.scans:
            frames,sid,subj,exp,hofid,gad=r['frames'],r['ID'],r['subject'],r['experiment'],r['hofid'],r['gad']  
            if exp in d: expd=d[exp]
            else: d[exp]={}; expd=d[exp]; add_default_vals(expd,def_keys)
            nt1c=expd['nT1c'] if 'nT1c' in expd else 0
            expd['subject']=subj
            if gad=="1" and hofid=='T1lo': add_val(expd,'T1c',sid); nt1c+=1
            elif hofid=='MPRAGE' or hofid=='T1hi': add_val(expd,'T1nc',sid)
            elif hofid=='T2hi': add_val(expd,'T2',sid)
            elif hofid=='T2FLAIR': add_val(expd,'T2FLAIR',sid)
            expd['nT1c']=nt1c
            
            if subj in confs: conf=confs[subj]
            else: 
                confs[subj]={}; conf=confs[subj]; conf['experiments']=[exp]; 
                conf['subject']=subj; conf['targ_experiment']=None
            if exp not in conf['experiments']: conf['experiments']+=[exp]
               
            if nt1c>0:
                if 'nT1c' not in conf: conf['nT1c']=nt1c; conf['targ_experiment']=exp
                elif conf['nT1c']<nt1c: conf['nT1c']=nt1c; conf['targ_experiment']=exp
            else:
                conf['nT1c']=0
                           
        #l1=list(d.values()); print(l1[:5]) 
        #l1=list(confs.values()); print(l1[:5])
        
        #now create run configurations, one configuration per subject.
        for conf_key in confs.keys():
            conf=confs[conf_key]
            add_default_vals(conf,def_keys)
            add_default_vals(conf,def_keys_imp)
            texp_name=conf['targ_experiment']
            if texp_name is not None:
                texp=d[texp_name]
                for k in def_keys: conf[k]=';'.join(texp[k]) if len(texp[k])>0 else 'NA'
                for k in def_keys_imp: conf[k]='NA'
            else: texp=None
            for exp in conf['experiments']: 
                if exp==texp_name: continue
                exp_dic=d[exp]
                for i in range(len(def_keys_imp)):
                    k,k1=def_keys[i],def_keys_imp[i]
                    l=[ exp+":"+kv for kv in exp_dic[k] if len(kv)>0 ]                                        
                    conf[k1]=';'.join(l) if len(l)>0 else 'NA'
                    
                '''
                for k in def_keys:
                    l=[ exp+":"+kv for kv in exp_dic[k] ]
                    l1=[conf[k]] if len(conf[k])>0 else []
                    l1=l1+l if len(l)>0 else l1
                    conf[k]='' if len(l1)<1 else ','.join(l1)
                '''
            #print('conf:',conf)

        '''                    
        exps=[]
        for key in d.keys():
            e=d[key]; e['experiment']=key
            add_default_vals(e,def_keys)
            exps+=[e]
            
        '''
        #print(exps[:20])
        fil='run_configurations.csv'
        vals=list(confs.values())
        print(vals[:5])
        self.write_scans_csv(list(confs.values()),fil)
        self.show_file_link(self._group_lnk,fil)        
    
    def collect_scans(self,b):
        subjects=[ s['Subject'] for s in self._rows ]
        self.scans=self._xi.list_scans_all(subjects,'',self._coll_status)
        self._coll_status='Status: found {} scans'.format(len(self.scans))
        fil='all_scans.csv'
        self.write_scans_csv(self.scans,fil)
        self.show_file_link(self._out_lnk,fil)
            
    def run_classifier(self,b):
        hofids=self.hof_classifier.infer_nn(self.scans)
        for i in range(len(self.scans)):
            self.scans[i]['hofid']=hofids[i]
        fil='all_scans_hofid.csv'
        self.write_scans_csv(self.scans,fil)
        self.show_file_link(self._classify_lnk,fil)
        
    def run_detect_gad(self,b):
        pass
    

In [7]:
xl=XNATLogin()
#xl.sp.project='IMIND_PILOT'
xl.sp.project='CONDR_METS'

scl=ScanClassifierLauncher(xl.sp)
pages=[
        {'title':'Login','frontdesk':xl,'plumbing':None,'prev_label':None,'next_label':'Configure & run'},
        {'title':'Classify scans','frontdesk':scl,'plumbing':None,'prev_label':'Login','next_label':None}
    ]
g=GUIBook(pages)

[{'experiments': ['IppolitoGBMSubj006_MR'], 'subject': 'IppolitoGBMSubj006', 'targ_experiment': None, 'nT1c': 0, 'T1c': [], 'T1nc': [], 'T2': [], 'T2FLAIR': [], 'T1c_imp': 'NA', 'T1nc_imp': 'IppolitoGBMSubj006_MR:3;IppolitoGBMSubj006_MR:1', 'T2_imp': 'NA', 'T2FLAIR_imp': 'NA'}, {'experiments': ['IppolitoGBMSubj012_MR'], 'subject': 'IppolitoGBMSubj012', 'targ_experiment': None, 'nT1c': 0, 'T1c': [], 'T1nc': [], 'T2': [], 'T2FLAIR': [], 'T1c_imp': 'NA', 'T1nc_imp': 'IppolitoGBMSubj012_MR:6;IppolitoGBMSubj012_MR:1;IppolitoGBMSubj012_MR:9;IppolitoGBMSubj012_MR:10;IppolitoGBMSubj012_MR:17;IppolitoGBMSubj012_MR:18', 'T2_imp': 'IppolitoGBMSubj012_MR:11;IppolitoGBMSubj012_MR:13;IppolitoGBMSubj012_MR:19;IppolitoGBMSubj012_MR:20', 'T2FLAIR_imp': 'NA'}, {'experiments': ['IppolitoGBMSubj015_MR_part1', 'IppolitoGBMSubj015_MR_part3', 'IppolitoGBMSubj015_MR_part2'], 'subject': 'IppolitoGBMSubj015', 'targ_experiment': 'IppolitoGBMSubj015_MR_part1', 'nT1c': 1, 'T1c': '26', 'T1nc': '2', 'T2': 'NA', 'T2F